# Загружаем и разбираемся с данными

In [847]:
import pandas as pd
import numpy as np
import re
import datetime
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt


df = pd.read_csv("C:/Users/enasy/Downloads/main_task_new.csv")


# Part 1: Fills empty values for Number of Reviews with zeros:
df['Number of Reviews'].fillna(0, inplace = True)


# Part 2: Changes $ symbols to numbers, assignes most common price range to NaN values:

def price_to_int(item):
    if item == '$':
        return int(1)
    elif item == '$$ - $$$':
        return int(2)
    elif item == '$$$$':
        return int(3)

df['Price Range'].update(df['Price Range'].apply(price_to_int))
df['Price Range'].fillna(df['Price Range'].median(), inplace = True)


# Part 3: Converts string data in Cuisine Style column to lists, replaces NaN with the most common cuisine, creates new
# column with number of cuisines:

def sanitize(item):
    return item.replace('\'', '').strip()

def to_list(str_item):
    if type(str_item) == str:
        return [sanitize(i) for i in str_item[1:-1].split(',')]

def str_to_list(s):
    return [s] if type(s) == str else s

# Creates dictionary for popularity of cuisines:
cuisine_dict = {}
for item in df['Cuisine Style']:
    if type(item) == str:
        cuisines = [sanitize(i) for i in item[1:-1].split(',')]
        for cuisine in cuisines:
            if cuisine not in cuisine_dict:
                cuisine_dict[cuisine] = 1
            else:
                cuisine_dict[cuisine] += 1

# Finds most popular cuisine in the created dictionary:
top_key = None
top_cuisine = 0
for key, value in cuisine_dict.items():
    if value > top_cuisine:
        top_cuisine = value
        top_key = key

# Convers data to lists and fills the most popular cuisine instead of NaN:
df['Cuisine Style'].update(df['Cuisine Style'].apply(to_list))
df['Cuisine Style'].fillna(top_key, inplace = True)
df['Cuisine Style'].update(df['Cuisine Style'].apply(str_to_list))

# Creates new column with number of cuisines:
df['Number of Cuisines'] = df['Cuisine Style'].apply(lambda x: len(x))
df = df.reindex(columns = ['Restaurant_id', 'City', 'Cuisine Style', 'Number of Cuisines', 'Ranking', 'Rating',
       'Price Range', 'Number of Reviews', 'Reviews', 'URL_TA', 'ID_TA'])


# Part 4: Extracts time for reviews and creates corresponding colunms, adds extra column for days between reviews:

def sanitize2(item):
    new_item = re.sub('[\'\[\]]', '', item)
    return new_item.strip()

def get_datetime_str(item):
    match = re.search('\d{2}/\d{2}/\d{4}', item)
    if match:
        return pd.to_datetime(match.group())
    return None

def to_dates_list(str_item):
    return [get_datetime_str(sanitize2(i)) for i in str_item.split(',')[-2:]]

df['Reviews'] = df['Reviews'].apply(lambda x: str(x))
review_dates = df['Reviews'].apply(to_dates_list)
review_dates_sep = pd.DataFrame(review_dates.tolist(), columns=['Review 1', 'Review 2'])
df['Review 1 time'] = review_dates_sep['Review 1']
df['Review 2 time'] = review_dates_sep['Review 2']
df['Time between reviews'] = df['Review 1 time'] - df['Review 2 time']


# Part 5: Normalizes time columns by time relevance:

def normalize(column):
    x = np.asmatrix(column)
    x_scaled = MinMaxScaler().fit_transform(x.T)
    return pd.DataFrame(x_scaled)

def time_relevance(column):
    last_review = column.max()
    time_relev = column.apply(lambda x: (last_review - x).total_seconds())
    time_relev.fillna(time_relev.mean(), inplace = True)
    return normalize(time_relev)

df['Review 1 time norm'] = time_relevance(df['Review 1 time'])
df['Review 2 time norm'] = time_relevance(df['Review 2 time'])

time_between_to_sec = df['Time between reviews'].apply(lambda x: x.total_seconds())
time_between_to_sec.fillna(time_between_to_sec.mean(), inplace = True)
df['Time between reviews norm'] = normalize(time_between_to_sec)

df = df.reindex(columns = ['Restaurant_id', 'City', 'Cuisine Style', 'Number of Cuisines', 'Ranking', 'Rating',
       'Price Range', 'Number of Reviews', 'Reviews', 'Review 1 time', 'Review 2 time', 'Time between reviews',
        'Review 1 time norm', 'Review 2 time norm', 'Time between reviews norm',
         'URL_TA', 'ID_TA'])

# Part 6: Creats dummy variables for Cuisine Styles:
def find_cuisine(cell):
    if item in cell:
        return 1
    return 0

cuisines = list(cuisine_dict.keys())
for item in cuisines:
    df[item] = df['Cuisine Style'].apply(find_cuisine)
    
    
# Part 7: Creates variables for vegeterian, healthy and not healthy selections:
df['Cuisine_vegan'] = df['Vegetarian Friendly'] + df['Vegan Options']   
df['Cuisine_healthy'] = df['Healthy'] + df['Gluten Free Options'] + df['Vegetarian Friendly'] + df['Vegan Options'] 
df['Cuisine_not_healthy']  = - df['Pizza'] - df['Street Food'] - df['Fast Food'] - df['Grill'] - df['Barbecue']
    
    
# Part 8: Sorts cities by geographical location and creates corresponding category columns:

def find_city(cell):
    if cell in geo[item]:
        return 1
    return 0

geo  = {'western': ['London','Paris','Berlin','Prague','Vienna','Amsterdam','Brussels','Hamburg',
'Minich','Lyon','Dublin','Edindurgh','Zurich','Geneva','Luxembourg'],
                 'easten': ['Budapest','Warsaw','Krakow','Bratislava','Ljubljana'],
                     'southen': ['Madrid','Barcelona','Milan','Rome','Lisbon','Athens','Oporto'],
                         'northen': ['Stockholm','Copenhagen','Oslo','Helsinki']}
for item in list(geo.keys()):
    df[item] = df['City'].apply(find_city)
    

# Part 9: Adds new features - number of restaurants in city, restaurant rank in city, and dummy cells for City:
df['N Restaurants in City'] = df.groupby(['City'])['Restaurant_id'].transform('count')
df['Rank for City'] = df['Ranking'] / df['N Restaurants in City']
cities = df['City']
df = pd.get_dummies(df, columns=[ 'City',], dummy_na=True)    
df['City'] = cities
 
    
# Part 10 Calculates average number of reviews by city and writes to new variable: 

#Number of reviews by city and number of restaurants with reviews
Number_of_Reviews_Sum = df.groupby(['City'])['Number of Reviews'].sum().sort_values(ascending=False)
Number_of_Reviews_Count = df.groupby(['City'])['Number of Reviews'].count().sort_values(ascending=False)
    
#Average number of reviews by city
medium_reviews_dict = {}
for city in list(df.City.unique()):
    medium_reviews_dict[city] = round(Number_of_Reviews_Sum[city]/Number_of_Reviews_Count[city])
        
#New variable showing average number of reviews by city for restaurants
df['Medium Reviews for City'] = df['City'].map(medium_reviews_dict)
    
    
# Part 11: Extracts keywords from Reviews and give score base on that:

def find_keyword(cell):
    score = 0
    lowercase_cell = cell.lower()
    for word in review_keywords['good']:
        if word in lowercase_cell:
            score += 2
            
    for word in review_keywords['bad']:
        if word in lowercase_cell:
            score -= -1
            
    return score

review_keywords = {'good': ['great','good','best','unique','delicious','exceptional','nice','wonderful',
                            'special','outstanding','delightful','tasty','perfect','awesome','brilliant'],
                    'bad': ['pricey','pricy','nothing','not ok','shameful','awful','avoid','overpriced',
                             'dreadful','disappointed','dishonest','terrible','poor','worst','poison','horrible',
                              'rude','soggy','appalling','scam','ripped off','disgusting']}
df['Reviews Score'] = df['Reviews'].apply(find_keyword)


# Part 12: Brings some external data (average income in city, population of city):
ext_data = pd.read_excel('C:/Users/enasy/Downloads/Cities_external_data.xlsx')
ext_data.columns = ['City', 'Monthly Income, USD', 'City Population']
df = df.merge(ext_data, on = 'City')


# Part 13 (optional): Checks for outliers with boxpot visualization, removes outliers in 'Ranking' column by IQR method: 

def outliers_iqr(column):
    quartile_1, quartile_3 = np.percentile(list(column), [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return df[(column < upper_bound)]
    
#plt.figure(figsize=(6,3))
#print('Ranking:', plt.boxplot(df['Ranking']))
#plt.figure(figsize=(6,3))
#print('Rating:', plt.boxplot(df['Rating']))
#plt.figure(figsize=(6,3))
#print('Number of Reviews:', plt.boxplot(df['Number of Reviews']))
#df = outliers_iqr(df['Ranking'])


# Part 14: Builds correlation table for quantitative features:

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

corr = df[['Number of Cuisines','Cuisine_healthy','Cuisine_not_healthy',
          'Ranking','Rating','Price Range', 'Number of Reviews',
          'Review 1 time norm', 'Review 2 time norm', 
          'Time between reviews norm','Reviews Score','N Restaurants in City',
          'Medium Reviews for City','Rank for City', 'Monthly Income, USD',
          'City Population']].\
        corr()

cmap = sns.diverging_palette(5, 250, as_cmap=True)

display(corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify()))


,Number of Cuisines,Cuisine_healthy,Cuisine_not_healthy,Ranking,Rating,Price Range,Number of Reviews,Review 1 time norm,Review 2 time norm,Time between reviews norm,Reviews Score,N Restaurants in City,Medium Reviews for City,Rank for City,"Monthly Income, USD",City Population
Number of Cuisines,1.00,0.53,-0.26,-0.32,0.12,0.00,0.41,-0.30,-0.36,-0.24,0.22,-0.02,0.14,-0.54,0.01,0.04
Cuisine_healthy,0.53,1.00,-0.01,-0.24,0.14,0.13,0.35,-0.24,-0.26,-0.17,0.10,0.01,0.09,-0.41,0.05,0.08
Cuisine_not_healthy,-0.26,-0.01,1.00,0.03,0.02,0.21,-0.06,0.06,0.08,0.05,-0.05,0.00,-0.14,0.05,0.01,-0.04
Ranking,-0.32,-0.24,0.03,1.00,-0.37,-0.03,-0.22,0.18,0.19,0.10,-0.13,0.71,0.11,0.55,0.23,0.58
Rating,0.12,0.14,0.02,-0.37,1.00,-0.03,0.03,-0.09,-0.10,-0.05,0.07,-0.06,0.05,-0.54,-0.03,-0.02
Price Range,0.00,0.13,0.21,-0.03,-0.03,1.00,0.11,0.00,0.00,-0.01,-0.05,0.01,-0.09,-0.05,0.07,-0.02
Number of Reviews,0.41,0.35,-0.06,-0.22,0.03,0.11,1.00,-0.20,-0.22,-0.15,0.12,0.03,0.18,-0.41,-0.04,0.02
Review 1 time norm,-0.30,-0.24,0.06,0.18,-0.09,0.00,-0.20,1.00,0.69,0.22,-0.02,-0.02,-0.05,0.33,-0.03,-0.04
Review 2 time norm,-0.36,-0.26,0.08,0.19,-0.10,0.00,-0.22,0.69,1.00,0.62,-0.06,-0.03,-0.07,0.37,-0.03,-0.07
Time between reviews norm,-0.24,-0.17,0.05,0.10,-0.05,-0.01,-0.15,0.22,0.62,1.00,-0.01,-0.03,-0.05,0.21,-0.03,-0.06


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [848]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'City', 'Cuisine Style', 'Rating', 'Reviews', 'Review 1 time', 'Review 2 time',
              'Time between reviews', 'URL_TA', 'ID_TA'], axis = 1)
y = df['Rating']

In [849]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [850]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [851]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [852]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [853]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.198633
